In [1]:
from dao.mongodbdao import MongoDBDao
from tfidf.TF_IDF import TF_IDF
import numpy as np 

dao = MongoDBDao({
    "host": "127.0.0.1",
    "db": "default",
    "wordindex_collection": "wordindex",
    "pagedetails_collection": "pagedetails",
    "username": "root",
    "password": "spezanw"
})

In [2]:
tfidf = TF_IDF(dao)

TF-IDF module: initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.276386.php'} contains no words
TF-IDF module: initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.461948.php'} contains no words
TF-IDF module: initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.587702.php'} contains no words
TF-IDF module: initial TF calculation: article with url {'https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.85381.php'} contains no words


# diverse Tests

In [3]:
a = [[1, 2],[3, 4]]
for j in a:
    print(j)

[1, 2]
[3, 4]


In [4]:
term_count = dao.getAllWordsWithCounts()
print("droge" in term_count)

True


In [5]:
#tfidf.getResultsForInput(user_input=["fußgängerin", "gestern", "nachmittag", "polizei", "waffe", "drogen", "fahrrad"], n_results=100, return_sims=True)
#tfidf.tf_idfs["https://www.berlin.de/polizei/polizeimeldungen/pressemitteilung.247355.php"]

tfidf.getResultsForInput(user_input=["drogen"], n_results=100, return_sims=True)



TF-IDF module: user input TF-IDF calculation: word {'drogen'} in query is not in db


[]

TF: 
- für jedes Dokument alle Wörter

IDF:
- Anzahl der Dokumente
- alle Wörter
- in wie vielen Dokumenten kommt jedes Wort vor?

1. Index: Anzahl der Dokumente
2. Inverser Index: Für term t get t.length

In [6]:
l = [1, 2, 3, 3]
l.append([])
l.append(3)
l

[1, 2, 3, 3, [], 3]

In [22]:
from pymongo import MongoClient
conf = {
    "host": "127.0.0.1",
    "port": 27018,
    "db": "default",
    "test_collection": "test",
    "username": "tfidf",
    "password": "tfidf",
    "authSource": "default"
}

db_name = conf.pop("db")
test_collection_name = conf.pop("test_collection")

client = MongoClient(**conf)
db = client[db_name]
test_collection = db[test_collection_name]

In [23]:
doc = {
            "_id": 0,
            "url": "abc",
            "title": "defg"
        }

test_collection.find_one_and_replace({"_id": doc["_id"]}, doc, upsert=True)

In [19]:
doc = {
            "_id": 1,
            "url": "abc"
        }
test_collection.find_one_and_replace({"_id": doc["_id"]}, doc, upsert=True)

OperationFailure: Authentication failed.

In [16]:
list(test_collection.find({}))

[{'_id': 0, 'url': 'abc', 'title': 'defg'}, {'_id': 1, 'url': 'abc'}]

(array([], dtype=float64), array([], dtype=int64))